# ViT: Vision Transformer
- 2020 구글에서 공개한, Transformer을 이미지 처리에 적용한 모델
- 이미지 데이터셋의 크기가 클 수록 더욱 좋은 성능을 선보임
- Convolution layer 없이 순수 Attention layer로만 모델을 학습하여 좋은 결과를 보임
- 전반적으로 BERT와 매우 흡사한 구조

In [ ]:
import math
from random import *
import torch.backends
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Modeling
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import torchvision

from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

## 모델 구현
![](https://viso.ai/wp-content/uploads/2021/09/vision-transformer-vit.png)


In [ ]:
# 파라미터 설정
img_size = 224   # 입력 이미지 크기
patch_size = 16  # 하나의 패치 크기
num_patches = (img_size // patch_size) ** 2  # 패치의 개수
n_classes = 365  # 분류할 클래스 수

n_layers = 12  # 트랜스포머 레이어 수
n_heads = 8  # 멀티 헤드 어텐션에서의 헤드 수
d_model = 768  # 임베딩 차원
d_ff = d_model * 4  # 피드 포워드 네트워크의 차원
dropout = 0.1  # 드롭아웃 비율

lr = 1e-4   # 학습률 설정
batch_size = 16
epochs = 5  # 학습할 에폭 수

### 활성화 함수

GELU(Gaussian Error Linear Unit)라는 활성화 함수를 구현해봅니다. 

GELU는 BERT와 같은 트랜스포머 모델에서 자주 사용되며, 다른 활성화 함수보다 깊은 신경망에서 잘 동작하는 것으로 알려져 있습니다.

In [ ]:
def gelu(x):
    # Hugging Face에서 구현한 gelu 활성화 함수
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

### Patch Embedding layer 클래스 선언
![](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F7a096efc8f3cc40849ee17a546dc0e685da2dc73-4237x1515.png&w=3840&q=75)

1. Patch embedding
  
ViT에서는 이미지를 N x N 크기의 작은 패치로 분할합니다. 예를 들어, 224 x 224 크기의 이미지를 16 x 16 크기의 패치로 분할하면 총 196개의 패치가 생성됩니다. 각 패치의 크기는 16 x 16 x 3(RGB 3채널)을 하나의 벡터로 변환한 후, 이를 임베딩 벡터로 변환합니다. 각 패치는 토큰처럼 취급되며, BERT 모델에서 문장의 단어를 처리하는 방식과 유사합니다.  

2. [CLS] token
  
또한 BERT와 마찬가지로, ViT는 CLS 토큰을 사용합니다. 이 토큰은 패치들 앞에 추가되며, 최종적으로 이미지의 전체적인 표현을 얻고 분류를 하기 위해 사용됩니다.

3. Position embedding
  
트랜스포머 모델은 위치 정보를 내재적으로 처리하지 않기 때문에 패치의 순서 정보가 없으면 학습이 어려워집니다. 이를 해결하기 위해 각 패치에 대한 위치 임베딩이 추가됩니다. 즉, 각 패치가 이미지의 어느 위치에서 나온 것인지를 알려주는 정보가 포함됩니다.

In [ ]:
# 패치 임베딩 클래스 (이미지 패치를 입력받아 벡터로 변환)
class PatchEmbedding(nn.Module):
    def __init__(self, img_size, patch_size, d_model):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_dim = patch_size * patch_size * 3  # RGB 이미지이므로 3 채널
        self.projection = nn.Linear(self.patch_dim, d_model)  # 패치를 d_model 차원으로 변환
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))  # [CLS] 토큰 추가
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, d_model))  # 위치 임베딩
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 입력 이미지 x를 패치로 분할
        batch_size = x.shape[0]
        x = x.view(batch_size, 3, self.img_size, self.img_size)  # (batch_size, 3, img_size, img_size)
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)   # patch_size 대로 이미지를 분할
        x = x.permute(0, 2, 3, 1, 4, 5).contiguous().view(batch_size, -1, self.patch_dim)  # (batch_size, num_patches, patch_dim)

        # 패치 임베딩 적용
        x = self.projection(x)  # (batch_size, num_patches, d_model)

        # [CLS] 토큰을 입력에 추가
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)  # (batch_size, 1, d_model)
        x = torch.cat((cls_tokens, x), dim=1)  # (batch_size, num_patches + 1, d_model)

        # 위치 임베딩 추가
        x = x + self.pos_embedding
        return self.dropout(x)

### Multihead attention 클래스 선언

트랜스포머 계열의 모델에서 가장 중요한 부분을 꼽으라면, 망설임 없이 멀티 헤드 어텐션 메커니즘이라고 할 수 있습니다.  

멀티 헤드 어텐션은 트랜스포머 아키텍처의 핵심 구성 요소 중 하나로, 셀프 어텐션(Self-Attention) 메커니즘을 여러 번 동시에 수행하는 방식입니다.  

입력 데이터의 다양한 부분에 주목하여 정보를 집계하는 데 사용되며, 특히나 복잡한 패턴과 관계를 학습하는 데 매우 유용합니다

초기화 부분: 쿼리(Q), 키(K), 값(V)에 대한 선형 변환을 정의합니다. 이 변환은 입력 데이터를 여러 '헤드'로 분할하여 각 헤드에서 어텐션을 계산하는 데 사용됩니다.

순전파 부분:

1. 쿼리, 키, 값에 대한 선형 변환을 수행합니다.

2. 멀티 헤드 어텐션을 위해 데이터를 여러 헤드로 분할합니다.

3. 각 헤드에서 어텐션 스코어를 계산하고, 필요한 경우 마스크를 적용합니다.

4. 어텐션 가중치를 계산하고, 이를 사용하여 값 행렬과 곱하여 어텐션 출력을 얻습니다.
5. 모든 헤드의 출력을 연결하고, 추가적인 선형 변환을 수행합니다.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()

        # 쿼리, 키, 값에 대한 선형 변환
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        # 드롭아웃 적용
        self.dropout = nn.Dropout(dropout)

        # 멀티 헤드 어텐션 후의 선형 변환
        self.fc = nn.Linear(d_model, d_model)

        # 스케일링 팩터
        self.scale = torch.sqrt(torch.FloatTensor([d_model // n_heads])).to(device)
        # 소프트맥스 함수 정의
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]

        # 쿼리, 키, 값에 대한 선형 변환
        Q = self.w_q(query)
        K = self.w_k(key)
        V = self.w_v(value)


        # 멀티 헤드 어텐션을 위한 차원 변환
        Q = Q.view(batch_size, -1, n_heads, d_model // n_heads).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, n_heads, d_model // n_heads).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, n_heads, d_model // n_heads).permute(0, 2, 1, 3)

        # 어텐션 스코어 계산
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        # 마스크 적용
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        # 소프트맥스 함수를 통해 어텐션 가중치 계산
        attention = dropout(self.softmax(energy))


        # 어텐션 가중치와 값 행렬을 곱하여 출력 계산
        x = torch.matmul(attention, V)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(batch_size, -1, d_model)

        # 최종 선형 변환
        x = self.fc(x)

        return x

중간에 멀티 헤드 어텐션을 위해 차원을 변환하는 과정이 있습니다.

`permute()`는 텐서의 차원을 재배열하는 함수로, 여기서 `permute(0, 2, 1, 3)`를 사용하는 이유는 멀티 헤드 어텐션을 계산하기 위해 텐서의 차원을 적절하게 재배열하기 위함입니다.

원래 텐서의 차원은 `[batch_size, seq_len, n_heads, d_model // n_heads]`입니다. 여기서:

- `batch_size`: 배치 크기

- `seq_len`: 시퀀스 길이

- `n_heads`: 어텐션 헤드의 수

- `d_model`: 모델의 차원

permute(0, 2, 1, 3)를 사용하면 차원의 순서가 [batch_size, n_heads, seq_len, d_model // n_heads]로 변경됩니다.  

이렇게 차원을 재배열하면, 각 어텐션 헤드는 독립적으로 시퀀스에 대한 어텐션을 계산할 수 있습니다. 그리고 이렇게 계산된 결과를 다시 원래의 차원 순서로 변경하여 최종 결과를 얻을 수 있습니다.


### Positionwise Feedforward Network

PositionwiseFeedforward 네트워크는 트랜스포머 아키텍처의 각 인코더와 디코더 레이어에 포함되어 있습니다. 이 네트워크는 기본적으로 두 개의 선형 변환을 연속적으로 적용하는데, 여기서는 1D Convolution을 사용하여 이 변환을 수행합니다.

![](https://miro.medium.com/max/1906/1*1l5JbeGfEGh2oxjI8koHdQ.png)

초기화 부분: 두 개의 1D Conv 레이어를 정의합니다. 첫 번째 합성곱은 `d_model` 차원의 입력을 `d_ff` 차원으로 확장하고, 두 번째 합성곱은 그 결과를 다시 `d_model` 차원으로 축소합니다.

순전파 부분:

1. 입력 x의 차원을 변경하여 Convolution을 적용하기 적합한 형태로 만듭니다.

2. 첫 번째 합성곱 레이어와 GELU 활성화 함수를 적용한 후, 결과에 드롭아웃을 적용합니다.

3. 두 번째 Convolution 레이어를 적용합니다.

4. 결과의 차원을 원래대로 변경하여 출력합니다.

이 네트워크는 멀티헤드 어텐션의 출력에 비선형 변환을 추가하여 모델의 표현력을 높이는 역할을 합니다.

In [ ]:
class PositionwiseFeedforward(nn.Module):
    def __init__(self):
        super().__init__()

        # 1D Convolution을 사용하여 선형 변환을 수행
        self.fc1 = nn.Conv1d(d_model, d_ff, 1)
        self.fc2 = nn.Conv1d(d_ff, d_model, 1)

        # 드롭아웃 정의
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 입력 x의 차원을 변경
        x = x.permute(0, 2, 1)

        # 첫 번째 Convolution과 활성화 함수 적용 후 드롭아웃
        x = self.fc1(x)


        # 두 번째 Convolution 적용
        # Your code

        # 차원을 원래대로 변경하여 출력
        x = x.permute(0, 2, 1)

        return x

### Encoder layer

트랜스포머 아키텍처의 인코더 레이어를 구현한 클래스입니다.  

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_22_teJgoCi.max-800x600.png)

각 인코더 레이어는 앞서 선언한 두 가지 주요 구성 요소로 이루어져 있습니다:
- 멀티 헤드 셀프 어텐션

- Position-wise feedforward network

초기화 부분:

- `MultiHeadAttention`: 멀티헤드 셀프 어텐션을 수행합니다. 이는 입력 시퀀스 내의 각 토큰이 다른 모든 토큰과 어떻게 상호작용하는지를 파악합니다.

- `PositionwiseFeedforward`: 네트워크를 통해 추가적인 비선형 변환을 수행합니다.

- `LayerNorm`: 레이어 정규화는 각 레이어의 출력을 안정화하여 학습을 도와줍니다.

- `Dropout`: 과적합을 방지하기 위한 드롭아웃입니다.

순전파 부분:

- 멀티헤드 셀프 어텐션을 적용한 후, 그 결과와 원래의 입력을 더하고 레이어 정규화를 수행합니다. (잔여 연결 및 레이어 정규화)

- Position-wise Feed forward를 적용한 후, 그 결과와 이전 단계의 출력을 더하고 다시 레이어 정규화를 수행합니다.

이 구조는 BERT 인코더의 각 레이어에서 반복적으로 사용되며, 여러 레이어를 쌓아 복잡한 패턴과 관계를 학습할 수 있게 합니다.


In [ ]:
# 트랜스포머 인코더 레이어 정의
class EncoderLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.attention = MultiHeadAttention()
        self.feed_forward = PositionwiseFeedforward()
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.layer_norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Self-attention
        # Your code
        
        # Add & normalization
        # Your code
        """
        1. dropout
        2. skip connection
        3. layer norm
        """
        
        # FFNN
        # Your code
        
        # Add & normalization
        # Your code
        """
        1. dropout
        2. skip connection
        3. layer norm
        """
        return 

### ViT 클래스 선언

위 내용들을 바탕으로 ViT 모델을 클래스로 선언하겠습니다.  

초기화 부분:

- `embedding`: 이미지가 패치 단위로 분할 후 임베딩을 거치며 위치 정보와 결합됩니다. [CLS]이 추가되며 벡터 형태로 변환하는 임베딩 레이어입니다.

- `encoder_layers`: ViT 모델의 핵심 부분인 인코더 레이어들의 리스트입니다. 각 레이어는 멀티헤드 어텐션과 Position-wise Feedforward 네트워크를 포함합니다.

- `linear, activn, norm`: 추가적인 변환을 위한 레이어와 활성화 함수입니다.

- classifier: 이미지의 특징을 읽고 클래스로 분류합니다.

순전파 부분:

- 입력 이미지는 `embedding` 레이어를 통과하여 임베딩 벡터로 변환됩니다.

- 임베딩 출력은 순차적으로 각 `encoder_layers`를 통과하며, 각 레이어에서는 멀티헤드 어텐션과 Position-wise Feedforward 연산이 수행됩니다.

- 모든 인코더 레이어를 통과한 후, `[CLS]` 토큰의 출력만을 사용하여 이미지를 분류합니다.

- 이 표현은 `classifier`를 통과하여 최종 출력을 생성합니다.

In [ ]:
# ViT 모델 정의
class ViT(nn.Module):
    def __init__(self, img_size, patch_size, d_model, n_layers, n_classes):
        super().__init__()
        self.embedding = PatchEmbedding(img_size, patch_size, d_model)
        self.encoder_layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.classifier = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, n_classes)
        )

    def forward(self, x):
        # 패치 임베딩 통과
        # Your code
        
        # 인코더 레이어 통과
        # Your code
            
        # [CLS] 토큰만 사용하여 최종 분류
        cls_token_output = x[:, 0]
        # Your code
        
        return 

In [ ]:
# 모델 생성
vit_model = ViT(img_size=img_size, 
                patch_size=patch_size, 
                d_model=d_model, 
                n_layers=n_layers, 
                n_classes=n_classes).to(device)
vit_model

In [ ]:
# 임의의 입력 이미지 (batch_size, 3, img_size, img_size) 생성
dummy_input = torch.randn(batch_size, 3, img_size, img_size)

output = vit_model(dummy_input.to(device))

output.shape

## Dataset

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),  # 이미지 크기 조정
    torchvision.transforms.ToTensor(),  # 이미지를 Tensor로 변환
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

In [ ]:
places365 = torchvision.datasets.Places365(root="./data", 
                                           download=True, 
                                           small=True,
                                           transform=transform,
                                           split="val"
                                           )

In [ ]:
len(places365.imgs), len(places365.targets)

In [ ]:
len(places365.classes)

In [ ]:
# 데이터셋을 10% 학습용, 5% 검증용으로 나누기
train_size = int(0.1 * len(places365))
val_size = int(train_size/2)
test_size = len(places365) - train_size - val_size
train_dataset, validation_dataset, test_dataset = random_split(places365, [train_size, val_size, test_size])

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch.optim as optim

# 손실 함수 정의 (CrossEntropyLoss)
criterion = nn.CrossEntropyLoss()

# 옵티마이저 정의 (Adam)
optimizer = optim.Adam(vit_model.parameters(), lr=0.001)

In [ ]:
def train_and_validate(model, train_dataloader, validation_dataloader, epochs, optimizer, criterion, device):
    # 전체 에포크만큼 학습 및 검증 반복
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        print('-' * 10)

        # 학습 모드 설정
        model.train()
        total_loss = 0
        train_preds, train_labels = [], []

        # 학습 데이터로 학습 진행
        progress_bar = tqdm(train_dataloader, desc='Training', position=0, leave=True)
        for step, batch in enumerate(progress_bar):
            # 배치 데이터를 디바이스에 할당 (이미지와 레이블)
            b_images = batch[0].to(device)
            b_labels = batch[1].to(device)

            # 그래디언트 초기화
            model.zero_grad()

            # 모델에 입력 데이터 전달 및 출력 얻기
            outputs = model(b_images)

            # 손실 계산 (다중 클래스 분류에 적합한 CrossEntropyLoss 사용)
            loss = criterion(outputs, b_labels)
            total_loss += loss.item()

            # 그래디언트 계산
            loss.backward()

            # 그래디언트 클리핑
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # 파라미터 업데이트
            optimizer.step()

            # 예측값 및 레이블 저장
            _, preds = torch.max(outputs, 1)  # 예측 클래스
            train_preds.extend(preds.detach().cpu().numpy().tolist())
            train_labels.extend(b_labels.detach().cpu().numpy().tolist())

            # 진행 바 업데이트
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

        # 평균 학습 손실 출력
        avg_train_loss = total_loss / len(train_dataloader)
        print("\nAverage training loss: {0:.2f}".format(avg_train_loss))

        # 학습 정확도 계산 및 출력
        train_acc = accuracy_score(train_labels, train_preds)
        print("Training Accuracy: {0:.2f}".format(train_acc))

        # 검증 모드 설정
        model.eval()
        total_eval_loss = 0
        val_preds, val_labels = [], []

        # 검증 데이터로 검증 진행
        progress_bar = tqdm(validation_dataloader, desc='Validation', position=0, leave=True)
        for batch in progress_bar:
            # 배치 데이터를 디바이스에 할당
            b_images = batch[0].to(device)
            b_labels = batch[1].to(device)

            # 그래디언트 계산 비활성화
            with torch.no_grad():
                # 모델에 입력 데이터 전달 및 출력 얻기
                outputs = model(b_images)

            # 손실 계산
            loss = criterion(outputs, b_labels)
            total_eval_loss += loss.item()

            # 예측값 및 레이블 저장
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.detach().cpu().numpy().tolist())
            val_labels.extend(b_labels.detach().cpu().numpy().tolist())

            # 진행 바 업데이트
            progress_bar.set_postfix({'validation_loss': '{:.3f}'.format(loss.item()/len(batch))})

        # 평균 검증 손실 출력
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        print("\nAverage validation loss: {0:.2f}".format(avg_val_loss))

        # 검증 정확도 계산 및 출력
        val_acc = accuracy_score(val_labels, val_preds)
        print("Validation Accuracy: {0:.2f}".format(val_acc))

In [ ]:
# 학습 시작
train_and_validate(vit_model, train_loader, val_loader, epochs, optimizer, criterion, device)